In [1]:
# Install required packages (run this cell first if packages are not installed)
# !pip install folium ipywidgets

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette('husl')


In [2]:
# Load the rainfall data
df = pd.read_csv('Rainfall_Data_LL.csv')

# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()


Dataset shape: (4187, 22)

Column names:
['Name', 'SUBDIVISION', 'YEAR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANNUAL', 'Jan-Feb', 'Mar-May', 'June-September', 'Oct-Dec', 'Latitude', 'Longitude']

First few rows:


,Name,SUBDIVISION,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,...,OCT,NOV,DEC,ANNUAL,Jan-Feb,Mar-May,June-September,Oct-Dec,Latitude,Longitude
0,ROW1,Andaman & Nicobar Islands,1901,49.2,87.1,29.2,2.3,528.8,517.5,365.1,...,388.5,558.2,33.6,3373.2,136.3,560.3,1696.3,980.3,12.61124,92.83165
1,ROW2,Andaman & Nicobar Islands,1902,0.0,159.8,12.2,0.0,446.1,537.1,228.9,...,197.2,359.0,160.5,3520.7,159.8,458.3,2185.9,716.7,12.61124,92.83165
2,ROW3,Andaman & Nicobar Islands,1903,12.7,144.0,0.0,1.0,235.1,479.9,728.4,...,181.2,284.4,225.0,2957.4,156.7,236.1,1874.0,690.6,12.61124,92.83165
3,ROW4,Andaman & Nicobar Islands,1904,9.4,14.7,0.0,202.4,304.5,495.1,502.0,...,222.2,308.7,40.1,3079.6,24.1,506.9,1977.6,571.0,12.61124,92.83165
4,ROW5,Andaman & Nicobar Islands,1905,1.3,0.0,3.3,26.9,279.5,628.7,368.7,...,260.7,25.4,344.7,2566.7,1.3,309.7,1624.9,630.8,12.61124,92.83165


In [3]:
# Data cleaning and preparation
print("Data types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())

# Check for unique years and subdivisions
print(f"\nYears available: {df['YEAR'].min()} to {df['YEAR'].max()}")
print(f"Number of subdivisions: {df['SUBDIVISION'].nunique()}")
unique_locations = df[['Latitude', 'Longitude']].drop_duplicates()
print(f"Number of unique locations: {unique_locations.shape[0]}")

# Basic statistics for annual rainfall
print("\nAnnual rainfall statistics:")
df['ANNUAL'].describe()


Data types:
Name               object
SUBDIVISION        object
YEAR                int64
JAN               float64
FEB               float64
MAR               float64
APR               float64
MAY               float64
JUN               float64
JUL               float64
AUG               float64
SEP               float64
OCT               float64
NOV               float64
DEC               float64
ANNUAL            float64
Jan-Feb           float64
Mar-May           float64
June-September    float64
Oct-Dec           float64
Latitude          float64
Longitude         float64
dtype: object

Missing values:
Name              0
SUBDIVISION       0
YEAR              0
JAN               0
FEB               0
MAR               0
APR               0
MAY               0
JUN               0
JUL               0
AUG               0
SEP               0
OCT               0
NOV               0
DEC               0
ANNUAL            0
Jan-Feb           0
Mar-May           0
June-September    0
Oct-D

count    4187.000000
mean     1414.591631
std       905.706627
min        62.200000
25%       804.950000
50%      1123.300000
75%      1650.300000
max      6331.100000
Name: ANNUAL, dtype: float64

In [4]:
# Function to determine rain emoji based on rainfall amount
def get_rain_emoji(rainfall):
    """
    Returns appropriate rain emoji based on rainfall amount (mm)
    """
    if rainfall < 100:
        return "🌤️"  # Light rain/partly cloudy
    elif rainfall < 500:
        return "🌦️"  # Light to moderate rain
    elif rainfall < 1000:
        return "🌧️"  # Moderate rain
    elif rainfall < 2000:
        return "⛈️"  # Heavy rain/thunderstorm
    else:
        return "🌊"  # Very heavy rain/flooding

# Function to get color based on rainfall intensity
def get_rain_color(rainfall):
    """
    Returns color based on rainfall intensity
    """
    if rainfall < 100:
        return '#87CEEB'  # Light blue
    elif rainfall < 500:
        return '#4682B4'  # Steel blue
    elif rainfall < 1000:
        return '#1E90FF'  # Dodger blue
    elif rainfall < 2000:
        return '#0000FF'  # Blue
    else:
        return '#000080'  # Navy blue

# Test the functions
print("Rain emoji examples:")
for rainfall in [50, 250, 750, 1500, 2500]:
    print(f"{rainfall}mm: {get_rain_emoji(rainfall)}")


Rain emoji examples:
50mm: 🌤️
250mm: 🌦️
750mm: 🌧️
1500mm: ⛈️
2500mm: 🌊


In [5]:
# Function to create interactive map for a specific year
def create_rainfall_map(year):
    """
    Creates a folium map showing rainfall data for a specific year
    """
    # Filter data for the specified year
    year_data = df[df['YEAR'] == year].copy()
    
    if year_data.empty:
        print(f"No data available for year {year}")
        return None
    
    # Create base map centered on India
    m = folium.Map(
        location=[20.5937, 78.9629],  # Center of India
        zoom_start=5,
        tiles='OpenStreetMap'
    )
    
    # Add title
    title_html = f'''
                 <h3 align="center" style="font-size:20px"><b>Rainfall Data for {year}</b></h3>
                 <p align="center">🌤️ < 100mm | 🌦️ 100-500mm | 🌧️ 500-1000mm | ⛈️ 1000-2000mm | 🌊 > 2000mm</p>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add markers for each location
    for idx, row in year_data.iterrows():
        lat, lon = row['Latitude'], row['Longitude']
        rainfall = row['ANNUAL']
        subdivision = row['SUBDIVISION']
        
        # Get emoji and color based on rainfall
        emoji = get_rain_emoji(rainfall)
        color = get_rain_color(rainfall)
        
        # Create popup text with detailed information
        popup_text = f"""
        <b>{subdivision}</b><br>
        Year: {year}<br>
        Annual Rainfall: {rainfall:.1f}mm<br>
        Rain Level: {emoji}
        """
        
        # Add circle marker
        folium.CircleMarker(
            location=[lat, lon],
            radius=max(5, min(20, rainfall/200)),  # Scale radius based on rainfall
            popup=folium.Popup(popup_text, max_width=300),
            color='black',
            weight=1,
            fillColor=color,
            fillOpacity=0.7,
            tooltip=f"{subdivision}: {rainfall:.1f}mm {emoji}"
        ).add_to(m)
        
        # Add emoji marker on top
        folium.Marker(
            location=[lat, lon],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 20px; text-align: center;">{emoji}</div>',
                icon_size=(30, 30),
                icon_anchor=(15, 15),
            )
        ).add_to(m)
    
    return m

# Test with a specific year
print("Creating map for year 2000...")
test_map = create_rainfall_map(2000)
if test_map:
    print("Map created successfully! Display it in the next cell.")


Creating map for year 2000...
Map created successfully! Display it in the next cell.


In [6]:
# Interactive widget with time slider
def interactive_rainfall_map():
    """
    Creates an interactive rainfall map with time slider
    """
    # Get available years
    years = sorted(df['YEAR'].unique())
    
    # Create year slider
    year_slider = widgets.IntSlider(
        value=years[0],
        min=years[0],
        max=years[-1],
        step=1,
        description='Year:',
        style={'description_width': 'initial'},
        continuous_update=False
    )
    
    # Create play button for animation
    play_button = widgets.Play(
        value=years[0],
        min=years[0],
        max=years[-1],
        step=1,
        description="Press play",
        disabled=False
    )
    
    # Link play button to slider
    widgets.jslink((play_button, 'value'), (year_slider, 'value'))
    
    # Create output widget for the map
    output = widgets.Output()
    
    # Function to update map based on year
    def update_map(year):
        with output:
            clear_output(wait=True)
            map_obj = create_rainfall_map(year)
            if map_obj:
                display(map_obj)
    
    # Connect slider to update function
    year_slider.observe(lambda change: update_map(change['new']), names='value')
    
    # Display initial map
    update_map(years[0])
    
    # Create and display the widget interface
    controls = widgets.HBox([play_button, year_slider])
    full_widget = widgets.VBox([controls, output])
    
    return full_widget

# Create the interactive widget
print("Creating interactive rainfall map with time slider...")
interactive_widget = interactive_rainfall_map()
print("Widget created! Display it in the next cell.")


Creating interactive rainfall map with time slider...


Widget created! Display it in the next cell.


In [7]:
# Display the interactive rainfall map
# Run this cell to see the interactive map with time slider
display(interactive_widget)

# Instructions:
# 1. Use the year slider to navigate through different years
# 2. Click the play button to animate through the years automatically
# 3. Click on any marker to see detailed rainfall information
# 4. Hover over markers to see quick information


In [8]:
# Additional analysis: Seasonal rainfall patterns
def create_seasonal_rainfall_map(year, season):
    """
    Creates a map showing seasonal rainfall patterns
    Seasons: 'Jan-Feb', 'Mar-May', 'June-September', 'Oct-Dec'
    """
    # Filter data for the specified year
    year_data = df[df['YEAR'] == year].copy()
    
    if year_data.empty:
        print(f"No data available for year {year}")
        return None
    
    # Create base map
    m = folium.Map(
        location=[20.5937, 78.9629],
        zoom_start=5,
        tiles='OpenStreetMap'
    )
    
    # Add title
    title_html = f'''
                 <h3 align="center" style="font-size:20px"><b>Seasonal Rainfall ({season}) - {year}</b></h3>
                 <p align="center">Seasonal rainfall patterns across India</p>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add markers for each location
    for idx, row in year_data.iterrows():
        lat, lon = row['Latitude'], row['Longitude']
        rainfall = row[season]
        subdivision = row['SUBDIVISION']
        
        # Get emoji and color based on rainfall (scaled for seasonal data)
        emoji = get_rain_emoji(rainfall)
        color = get_rain_color(rainfall)
        
        # Create popup text
        popup_text = f"""
        <b>{subdivision}</b><br>
        Year: {year}<br>
        Season: {season}<br>
        Rainfall: {rainfall:.1f}mm<br>
        Rain Level: {emoji}
        """
        
        # Add circle marker
        folium.CircleMarker(
            location=[lat, lon],
            radius=max(3, min(15, rainfall/100)),
            popup=folium.Popup(popup_text, max_width=300),
            color='black',
            weight=1,
            fillColor=color,
            fillOpacity=0.7,
            tooltip=f"{subdivision}: {rainfall:.1f}mm {emoji}"
        ).add_to(m)
        
        # Add emoji marker
        folium.Marker(
            location=[lat, lon],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 16px; text-align: center;">{emoji}</div>',
                icon_size=(25, 25),
                icon_anchor=(12, 12),
            )
        ).add_to(m)
    
    return m

# Example: Create seasonal map for monsoon season
print("Creating seasonal analysis for monsoon season (June-September) in 2010...")
seasonal_map = create_seasonal_rainfall_map(2010, 'June-September')
if seasonal_map:
    print("Seasonal map created! You can display it using: display(seasonal_map)")


Creating seasonal analysis for monsoon season (June-September) in 2010...
Seasonal map created! You can display it using: display(seasonal_map)
